# Evaluation Demo

This notebook demonstrates comprehensive evaluation analysis of the FHE NLP project results.

## Overview
- Load evaluation results from `data/results/evaluation.json`
- Visualize performance metrics (accuracy, precision, recall, F1-score)
- Plot privacy-utility trade-off curves (ε vs accuracy)
- Analyze security and privacy metrics
- Summarize best configurations and recommendations

## Key Features
- **Performance Analysis**: Comprehensive ML metrics visualization
- **Privacy-Utility Trade-offs**: Differential privacy parameter analysis
- **Security Assessment**: FHE security metrics evaluation
- **Configuration Optimization**: Best parameter recommendations

---

In [ ]:
# Import required libraries
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
from scipy.interpolate import make_interp_spline
from scipy.stats import pearsonr

# Set up plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

# Configure matplotlib for better plots
plt.rcParams.update({
    'figure.figsize': (12, 8),
    'font.size': 11,
    'axes.titlesize': 14,
    'axes.labelsize': 12,
    'xtick.labelsize': 10,
    'ytick.labelsize': 10,
    'legend.fontsize': 10,
    'figure.titlesize': 16
})

print("📚 All imports successful!")
print("🎨 Plotting style configured")

## 1. Load Evaluation Results

Load the comprehensive evaluation results from the pipeline execution.

In [ ]:
# Load evaluation results
evaluation_file = Path('../data/results/evaluation.json')

if evaluation_file.exists():
    print(f"📂 Loading evaluation results from: {evaluation_file}")
    with open(evaluation_file, 'r', encoding='utf-8') as f:
        evaluation_data = json.load(f)
    print("✅ Evaluation data loaded successfully!")
else:
    print("⚠️ Evaluation file not found. Creating synthetic evaluation data for demonstration...")
    
    # Create comprehensive synthetic evaluation data
    evaluation_data = {
        "performance_metrics": {
            "accuracy": 0.847,
            "precision": 0.823,
            "recall": 0.871,
            "f1_score": 0.846,
            "roc_auc": 0.912,
            "classification_report": {
                "diabetes": {"precision": 0.85, "recall": 0.82, "f1-score": 0.84, "support": 45},
                "hypertension": {"precision": 0.81, "recall": 0.89, "f1-score": 0.85, "support": 52},
                "heart_disease": {"precision": 0.79, "recall": 0.88, "f1-score": 0.83, "support": 38},
                "asthma": {"precision": 0.88, "recall": 0.85, "f1-score": 0.86, "support": 41},
                "arthritis": {"precision": 0.82, "recall": 0.91, "f1-score": 0.86, "support": 47}
            }
        },
        "privacy_metrics": {
            "k_anonymity": {"k_value": 5, "privacy_level": "high"},
            "l_diversity": {"l_value": 3, "privacy_level": "medium"},
            "information_leakage": {"leakage_score": 0.12, "privacy_level": "high"},
            "differential_privacy": {"epsilon": 1.0, "privacy_level": "medium"},
            "overall_assessment": {"overall_privacy_level": "high", "privacy_score": 2.8}
        },
        "security_metrics": {
            "overall_security": {"overall_security_score": 87.3},
            "attack_resistance": {"overall_attack_resistance_score": 84.7},
            "key_security": {"key_security_strength": 128},
            "noise_analysis": {"initial_noise_budget_bits": 412.5}
        },
        "privacy_utility_experiments": {
            "epsilon_values": [0.1, 0.5, 1.0, 2.0, 5.0],
            "accuracy_values": [0.721, 0.789, 0.847, 0.863, 0.871],
            "precision_values": [0.698, 0.765, 0.823, 0.841, 0.849],
            "recall_values": [0.743, 0.812, 0.871, 0.884, 0.892],
            "f1_values": [0.720, 0.788, 0.846, 0.862, 0.870]
        },
        "model_comparison": {
            "logistic_regression": {
                "clear_accuracy": 0.863,
                "fhe_accuracy": 0.847,
                "training_time": 2.34,
                "inference_time": 0.0023
            },
            "svm": {
                "clear_accuracy": 0.841,
                "fhe_accuracy": 0.829,
                "training_time": 4.67,
                "inference_time": 0.0156
            }
        }
    }
    print("✅ Synthetic evaluation data created for demonstration")

# Display basic information about the loaded data
print(f"\n📊 Evaluation Data Overview:")
print(f"   Available sections: {list(evaluation_data.keys())}")

# Extract key metrics for quick overview
if 'performance_metrics' in evaluation_data:
    perf = evaluation_data['performance_metrics']
    print(f"\n🎯 Performance Summary:")
    print(f"   Accuracy: {perf.get('accuracy', 'N/A'):.3f}")
    print(f"   Precision: {perf.get('precision', 'N/A'):.3f}")
    print(f"   Recall: {perf.get('recall', 'N/A'):.3f}")
    print(f"   F1-Score: {perf.get('f1_score', 'N/A'):.3f}")

if 'privacy_metrics' in evaluation_data:
    privacy = evaluation_data['privacy_metrics']
    overall_privacy = privacy.get('overall_assessment', {})
    print(f"\n🔒 Privacy Summary:")
    print(f"   Overall Level: {overall_privacy.get('overall_privacy_level', 'N/A')}")
    print(f"   Privacy Score: {overall_privacy.get('privacy_score', 'N/A')}")

if 'security_metrics' in evaluation_data:
    security = evaluation_data['security_metrics']
    print(f"\n🛡️ Security Summary:")
    print(f"   Overall Score: {security.get('overall_security', {}).get('overall_security_score', 'N/A')}")
    print(f"   Attack Resistance: {security.get('attack_resistance', {}).get('overall_attack_resistance_score', 'N/A')}")

## 2. Performance Metrics Visualization

Create comprehensive bar charts for accuracy, precision, recall, and F1-score.

In [ ]:
# Create performance metrics visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('FHE NLP Project - Performance Metrics Analysis', fontsize=16, fontweight='bold')

# Extract performance metrics
perf_metrics = evaluation_data.get('performance_metrics', {})

# 1. Overall Performance Metrics Bar Chart
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
metric_labels = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
values = [perf_metrics.get(metric, 0) for metric in metrics]
colors = ['#2E86AB', '#A23B72', '#F18F01', '#C73E1D', '#5E548E']

bars = ax1.bar(metric_labels, values, color=colors, alpha=0.8, edgecolor='white', linewidth=1.5)
ax1.set_title('Overall Performance Metrics', fontweight='bold')
ax1.set_ylabel('Score')
ax1.set_ylim(0, 1.0)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, value in zip(bars, values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
             f'{value:.3f}', ha='center', va='bottom', fontweight='bold')

# Add reference lines
ax1.axhline(y=0.8, color='green', linestyle='--', alpha=0.5, label='Good (0.8)')
ax1.axhline(y=0.9, color='blue', linestyle='--', alpha=0.5, label='Excellent (0.9)')
ax1.legend(loc='upper right')

# 2. Per-Class Performance Analysis
classification_report = perf_metrics.get('classification_report', {})
if classification_report:
    classes = []
    precisions = []
    recalls = []
    f1_scores = []
    
    for class_name, metrics in classification_report.items():
        if isinstance(metrics, dict) and 'precision' in metrics:
            classes.append(class_name)
            precisions.append(metrics.get('precision', 0))
            recalls.append(metrics.get('recall', 0))
            f1_scores.append(metrics.get('f1-score', 0))
    
    if classes:
        x_pos = np.arange(len(classes))
        width = 0.25
        
        bars1 = ax2.bar(x_pos - width, precisions, width, label='Precision', color='#2E86AB', alpha=0.8)
        bars2 = ax2.bar(x_pos, recalls, width, label='Recall', color='#A23B72', alpha=0.8)
        bars3 = ax2.bar(x_pos + width, f1_scores, width, label='F1-Score', color='#F18F01', alpha=0.8)
        
        ax2.set_title('Per-Class Performance Metrics', fontweight='bold')
        ax2.set_xlabel('Medical Conditions')
        ax2.set_ylabel('Score')
        ax2.set_xticks(x_pos)
        ax2.set_xticklabels(classes, rotation=45, ha='right')
        ax2.legend()
        ax2.grid(True, alpha=0.3, axis='y')
        ax2.set_ylim(0, 1.0)
else:
    ax2.text(0.5, 0.5, 'No per-class data available', ha='center', va='center',
             transform=ax2.transAxes, fontsize=14, alpha=0.5)
    ax2.set_title('Per-Class Performance Metrics', fontweight='bold')

# 3. Model Comparison (if available)
model_comparison = evaluation_data.get('model_comparison', {})
if model_comparison:
    models = list(model_comparison.keys())
    clear_accuracies = [model_comparison[model].get('clear_accuracy', 0) for model in models]
    fhe_accuracies = [model_comparison[model].get('fhe_accuracy', 0) for model in models]
    
    x_pos = np.arange(len(models))
    width = 0.35
    
    bars1 = ax3.bar(x_pos - width/2, clear_accuracies, width, label='Clear Model', 
                    color='skyblue', alpha=0.8)
    bars2 = ax3.bar(x_pos + width/2, fhe_accuracies, width, label='FHE Model', 
                    color='lightcoral', alpha=0.8)
    
    ax3.set_title('Model Comparison: Clear vs FHE', fontweight='bold')
    ax3.set_xlabel('Model Type')
    ax3.set_ylabel('Accuracy')
    ax3.set_xticks(x_pos)
    ax3.set_xticklabels([model.replace('_', ' ').title() for model in models])
    ax3.legend()
    ax3.grid(True, alpha=0.3, axis='y')
    ax3.set_ylim(0, 1.0)
    
    # Add value labels
    for bars in [bars1, bars2]:
        for bar in bars:
            height = bar.get_height()
            ax3.text(bar.get_x() + bar.get_width()/2, height + 0.01,
                     f'{height:.3f}', ha='center', va='bottom', fontweight='bold')
else:
    ax3.text(0.5, 0.5, 'No model comparison data available', ha='center', va='center',
             transform=ax3.transAxes, fontsize=14, alpha=0.5)
    ax3.set_title('Model Comparison: Clear vs FHE', fontweight='bold')

# 4. Performance Distribution (Radar Chart)
if len(values) >= 4:  # Need at least 4 metrics for radar chart
    # Create radar chart
    angles = np.linspace(0, 2 * np.pi, len(metric_labels), endpoint=False).tolist()
    values_radar = values + values[:1]  # Complete the circle
    angles += angles[:1]
    
    ax4 = plt.subplot(2, 2, 4, projection='polar')
    ax4.plot(angles, values_radar, 'o-', linewidth=2, color='#2E86AB', alpha=0.8)
    ax4.fill(angles, values_radar, alpha=0.25, color='#2E86AB')
    ax4.set_xticks(angles[:-1])
    ax4.set_xticklabels(metric_labels)
    ax4.set_ylim(0, 1)
    ax4.set_yticks([0.2, 0.4, 0.6, 0.8, 1.0])
    ax4.set_yticklabels(['0.2', '0.4', '0.6', '0.8', '1.0'])
    ax4.grid(True, alpha=0.3)
    ax4.set_title('Performance Radar Chart', fontweight='bold', pad=20)
    
    # Add value labels
    for angle, value, label in zip(angles[:-1], values, metric_labels):
        ax4.text(angle, value + 0.05, f'{value:.2f}', ha='center', va='center', 
                fontweight='bold', fontsize=9)
else:
    ax4.text(0.5, 0.5, 'Insufficient data for radar chart', ha='center', va='center',
             transform=ax4.transAxes, fontsize=14, alpha=0.5)
    ax4.set_title('Performance Radar Chart', fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed performance analysis
print("📊 Detailed Performance Analysis:")
print("=" * 50)
print(f"🎯 Overall Performance:")
for metric, label, value in zip(metrics, metric_labels, values):
    if value > 0:
        performance_level = "Excellent" if value >= 0.9 else "Good" if value >= 0.8 else "Fair" if value >= 0.7 else "Needs Improvement"
        print(f"   {label}: {value:.4f} ({performance_level})")

if classification_report and classes:
    print(f"\n📋 Per-Class Analysis:")
    for i, class_name in enumerate(classes):
        print(f"   {class_name.title()}:")
        print(f"      Precision: {precisions[i]:.3f}")
        print(f"      Recall: {recalls[i]:.3f}")
        print(f"      F1-Score: {f1_scores[i]:.3f}")

if model_comparison:
    print(f"\n🤖 Model Comparison:")
    for model in models:
        clear_acc = model_comparison[model].get('clear_accuracy', 0)
        fhe_acc = model_comparison[model].get('fhe_accuracy', 0)
        accuracy_loss = clear_acc - fhe_acc
        print(f"   {model.replace('_', ' ').title()}:")
        print(f"      Clear Accuracy: {clear_acc:.4f}")
        print(f"      FHE Accuracy: {fhe_acc:.4f}")
        print(f"      Accuracy Loss: {accuracy_loss:.4f} ({accuracy_loss/clear_acc*100:.2f}%)")

## 3. Privacy-Utility Trade-off Analysis

Plot the privacy-utility curve showing the relationship between ε (epsilon) and accuracy.

In [ ]:
# Create privacy-utility trade-off visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Privacy-Utility Trade-off Analysis', fontsize=16, fontweight='bold')

# Extract privacy-utility experiment data
privacy_utility = evaluation_data.get('privacy_utility_experiments', {})
epsilon_values = privacy_utility.get('epsilon_values', [0.1, 0.5, 1.0, 2.0, 5.0])
accuracy_values = privacy_utility.get('accuracy_values', [0.721, 0.789, 0.847, 0.863, 0.871])
precision_values = privacy_utility.get('precision_values', [0.698, 0.765, 0.823, 0.841, 0.849])
recall_values = privacy_utility.get('recall_values', [0.743, 0.812, 0.871, 0.884, 0.892])
f1_values = privacy_utility.get('f1_values', [0.720, 0.788, 0.846, 0.862, 0.870])

# 1. Main Privacy-Utility Curve (ε vs Accuracy)
ax1.semilogx(epsilon_values, accuracy_values, 'o-', linewidth=3, markersize=8, 
             color='#2E86AB', alpha=0.8, label='Accuracy')
ax1.set_title('Privacy-Utility Trade-off Curve\n(ε vs Accuracy)', fontweight='bold')
ax1.set_xlabel('Epsilon (ε) - Privacy Budget', fontweight='bold')
ax1.set_ylabel('Accuracy', fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0.6, 1.0)

# Add privacy level annotations
privacy_regions = [
    (0.01, 0.1, 'Very High\nPrivacy', 'lightblue'),
    (0.1, 1.0, 'High\nPrivacy', 'lightgreen'),
    (1.0, 10.0, 'Medium\nPrivacy', 'lightyellow'),
    (10.0, 100.0, 'Low\nPrivacy', 'lightcoral')
]

y_min, y_max = ax1.get_ylim()
for x_min, x_max, label, color in privacy_regions:
    if x_min <= max(epsilon_values) and x_max >= min(epsilon_values):
        ax1.axvspan(x_min, x_max, alpha=0.1, color=color)
        ax1.text(np.sqrt(x_min * x_max), y_max - 0.02, label, ha='center', va='top', 
                fontsize=9, alpha=0.7, bbox=dict(boxstyle='round,pad=0.3', facecolor=color, alpha=0.3))

# Add value labels
for eps, acc in zip(epsilon_values, accuracy_values):
    ax1.annotate(f'{acc:.3f}', (eps, acc), textcoords="offset points", 
                xytext=(0,10), ha='center', fontweight='bold', fontsize=9)

# 2. All Metrics vs Epsilon
ax2.semilogx(epsilon_values, accuracy_values, 'o-', label='Accuracy', linewidth=2, markersize=6)
ax2.semilogx(epsilon_values, precision_values, 's-', label='Precision', linewidth=2, markersize=6)
ax2.semilogx(epsilon_values, recall_values, '^-', label='Recall', linewidth=2, markersize=6)
ax2.semilogx(epsilon_values, f1_values, 'd-', label='F1-Score', linewidth=2, markersize=6)

ax2.set_title('All Metrics vs Privacy Budget', fontweight='bold')
ax2.set_xlabel('Epsilon (ε)', fontweight='bold')
ax2.set_ylabel('Metric Score', fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0.6, 1.0)

# 3. Privacy Level Distribution
privacy_metrics = evaluation_data.get('privacy_metrics', {})
privacy_levels = []
privacy_labels = []

for metric, data in privacy_metrics.items():
    if isinstance(data, dict) and 'privacy_level' in data:
        privacy_levels.append(data['privacy_level'])
        privacy_labels.append(metric.replace('_', ' ').title())

if privacy_levels:
    level_counts = pd.Series(privacy_levels).value_counts()
    colors = {'high': '#2E86AB', 'medium': '#F18F01', 'low': '#E76F51'}
    pie_colors = [colors.get(level, '#808080') for level in level_counts.index]
    
    wedges, texts, autotexts = ax3.pie(level_counts.values, labels=level_counts.index,
                                       autopct='%1.1f%%', colors=pie_colors, startangle=90)
    ax3.set_title('Privacy Levels Distribution', fontweight='bold')
    
    for autotext in autotexts:
        autotext.set_color('white')
        autotext.set_fontweight('bold')
else:
    ax3.text(0.5, 0.5, 'No privacy level data available', ha='center', va='center',
             transform=ax3.transAxes, fontsize=14, alpha=0.5)
    ax3.set_title('Privacy Levels Distribution', fontweight='bold')

# 4. Utility Loss vs Privacy Gain
if len(accuracy_values) > 1:
    max_accuracy = max(accuracy_values)
    utility_loss = [(max_accuracy - acc) * 100 for acc in accuracy_values]  # Percentage loss
    privacy_gain = [100 / eps for eps in epsilon_values]  # Inverse of epsilon as privacy gain
    
    # Normalize privacy gain to 0-100 scale
    max_privacy_gain = max(privacy_gain)
    privacy_gain_normalized = [(pg / max_privacy_gain) * 100 for pg in privacy_gain]
    
    ax4.plot(privacy_gain_normalized, utility_loss, 'o-', linewidth=2, markersize=8, 
             color='#A23B72', alpha=0.8)
    ax4.set_title('Utility Loss vs Privacy Gain', fontweight='bold')
    ax4.set_xlabel('Privacy Gain (normalized)', fontweight='bold')
    ax4.set_ylabel('Utility Loss (%)', fontweight='bold')
    ax4.grid(True, alpha=0.3)
    
    # Add annotations for epsilon values
    for i, (pg, ul, eps) in enumerate(zip(privacy_gain_normalized, utility_loss, epsilon_values)):
        ax4.annotate(f'ε={eps}', (pg, ul), textcoords="offset points", 
                    xytext=(5,5), ha='left', fontsize=9, alpha=0.7)
else:
    ax4.text(0.5, 0.5, 'Insufficient data for utility loss analysis', ha='center', va='center',
             transform=ax4.transAxes, fontsize=14, alpha=0.5)
    ax4.set_title('Utility Loss vs Privacy Gain', fontweight='bold')

plt.tight_layout()
plt.show()

# Calculate and display trade-off analysis
print("🔒 Privacy-Utility Trade-off Analysis:")
print("=" * 50)

if len(epsilon_values) > 1 and len(accuracy_values) > 1:
    # Calculate correlation between log(epsilon) and accuracy
    log_epsilon = np.log10(epsilon_values)
    correlation, p_value = pearsonr(log_epsilon, accuracy_values)
    
    print(f"📈 Trade-off Statistics:")
    print(f"   Correlation (log ε, accuracy): {correlation:.4f}")
    print(f"   P-value: {p_value:.4f}")
    
    if correlation > 0.7:
        print(f"   Interpretation: Strong positive correlation - clear privacy-utility trade-off")
    elif correlation > 0.4:
        print(f"   Interpretation: Moderate correlation - noticeable trade-off")
    else:
        print(f"   Interpretation: Weak correlation - robust privacy protection")
    
    print(f"\n🎯 Epsilon Analysis:")
    for i, (eps, acc) in enumerate(zip(epsilon_values, accuracy_values)):
        privacy_level = "Very High" if eps <= 0.1 else "High" if eps <= 1.0 else "Medium" if eps <= 5.0 else "Low"
        utility_level = "Excellent" if acc >= 0.9 else "Good" if acc >= 0.8 else "Fair" if acc >= 0.7 else "Poor"
        print(f"   ε = {eps:>4}: Accuracy = {acc:.3f} | Privacy: {privacy_level:<9} | Utility: {utility_level}")
    
    # Find optimal epsilon (best balance)
    if len(accuracy_values) > 2:
        # Calculate efficiency score (accuracy per unit privacy cost)
        efficiency_scores = [acc / eps for acc, eps in zip(accuracy_values, epsilon_values)]
        best_idx = np.argmax(efficiency_scores)
        
        print(f"\n⭐ Optimal Configuration:")
        print(f"   Best ε: {epsilon_values[best_idx]}")
        print(f"   Accuracy: {accuracy_values[best_idx]:.4f}")
        print(f"   Efficiency Score: {efficiency_scores[best_idx]:.4f}")
        print(f"   Recommendation: Good balance between privacy and utility")

# Privacy level summary
if privacy_levels:
    print(f"\n🔐 Privacy Assessment Summary:")
    for level in ['high', 'medium', 'low']:
        count = privacy_levels.count(level)
        if count > 0:
            percentage = (count / len(privacy_levels)) * 100
            print(f"   {level.title()} Privacy: {count}/{len(privacy_levels)} metrics ({percentage:.1f}%)")
    
    overall_privacy = privacy_metrics.get('overall_assessment', {})
    if overall_privacy:
        print(f"   Overall Privacy Level: {overall_privacy.get('overall_privacy_level', 'N/A')}")
        print(f"   Privacy Score: {overall_privacy.get('privacy_score', 'N/A')}/3.0")

## 4. Security and Privacy Analysis

Analyze security metrics and privacy protection levels.

In [ ]:
# Security and Privacy Analysis
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Security and Privacy Analysis', fontsize=16, fontweight='bold')

# Extract security metrics
security_metrics = evaluation_data.get('security_metrics', {})
privacy_metrics = evaluation_data.get('privacy_metrics', {})

# 1. Security Scores Gauge Chart
overall_security = security_metrics.get('overall_security', {}).get('overall_security_score', 0)
attack_resistance = security_metrics.get('attack_resistance', {}).get('overall_attack_resistance_score', 0)

security_scores = [overall_security, attack_resistance]
security_labels = ['Overall Security', 'Attack Resistance']
colors = ['#2E86AB', '#A23B72']

bars = ax1.barh(security_labels, security_scores, color=colors, alpha=0.8)
ax1.set_title('Security Scores', fontweight='bold')
ax1.set_xlabel('Score (0-100)')
ax1.set_xlim(0, 100)
ax1.grid(True, alpha=0.3, axis='x')

# Add value labels
for bar, score in zip(bars, security_scores):
    ax1.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
             f'{score:.1f}', ha='left', va='center', fontweight='bold')

# Add security level indicators
for i, score in enumerate(security_scores):
    if score >= 90:
        level = "Excellent"
        color = 'green'
    elif score >= 80:
        level = "Very Good"
        color = 'blue'
    elif score >= 70:
        level = "Good"
        color = 'orange'
    else:
        level = "Needs Improvement"
        color = 'red'
    
    ax1.text(score/2, i, level, ha='center', va='center', 
             color='white', fontweight='bold', fontsize=10)

# 2. Privacy Metrics Comparison
privacy_scores = []
privacy_names = []

for metric, data in privacy_metrics.items():
    if isinstance(data, dict):
        if 'privacy_level' in data:
            level = data['privacy_level']
            score = 3 if level == 'high' else 2 if level == 'medium' else 1 if level == 'low' else 0
            privacy_scores.append(score)
            privacy_names.append(metric.replace('_', ' ').title())
        elif 'privacy_score' in data:
            privacy_scores.append(data['privacy_score'])
            privacy_names.append(metric.replace('_', ' ').title())

if privacy_scores and privacy_names:
    bars = ax2.bar(range(len(privacy_names)), privacy_scores, 
                   color=sns.color_palette("viridis", len(privacy_names)), alpha=0.8)
    ax2.set_title('Privacy Metrics Scores', fontweight='bold')
    ax2.set_xlabel('Privacy Metrics')
    ax2.set_ylabel('Privacy Score')
    ax2.set_xticks(range(len(privacy_names)))
    ax2.set_xticklabels(privacy_names, rotation=45, ha='right')
    ax2.grid(True, alpha=0.3, axis='y')
    
    # Add value labels
    for bar, score in zip(bars, privacy_scores):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
                 f'{score:.2f}', ha='center', va='bottom', fontweight='bold')
    
    # Add reference lines
    ax2.axhline(y=1, color='red', linestyle='--', alpha=0.5, label='Low')
    ax2.axhline(y=2, color='orange', linestyle='--', alpha=0.5, label='Medium')
    ax2.axhline(y=3, color='green', linestyle='--', alpha=0.5, label='High')
    ax2.legend()
else:
    ax2.text(0.5, 0.5, 'No privacy scores available', ha='center', va='center',
             transform=ax2.transAxes, fontsize=14, alpha=0.5)
    ax2.set_title('Privacy Metrics Scores', fontweight='bold')

# 3. Security vs Privacy Trade-off
if overall_security > 0 and privacy_scores:
    avg_privacy_score = np.mean(privacy_scores) if privacy_scores else 0
    
    # Create scatter plot
    ax3.scatter([overall_security], [avg_privacy_score * 33.33], s=200, 
               color='#F18F01', alpha=0.8, edgecolors='black', linewidth=2)
    
    ax3.set_title('Security vs Privacy Trade-off', fontweight='bold')
    ax3.set_xlabel('Security Score')
    ax3.set_ylabel('Privacy Score (normalized)')
    ax3.set_xlim(0, 100)
    ax3.set_ylim(0, 100)
    ax3.grid(True, alpha=0.3)
    
    # Add quadrant labels
    ax3.text(25, 75, 'High Privacy\nLow Security', ha='center', va='center', 
             bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.5))
    ax3.text(75, 75, 'High Privacy\nHigh Security', ha='center', va='center',
             bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))
    ax3.text(25, 25, 'Low Privacy\nLow Security', ha='center', va='center',
             bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.5))
    ax3.text(75, 25, 'Low Privacy\nHigh Security', ha='center', va='center',
             bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.5))
    
    # Add current position annotation
    ax3.annotate(f'Current\nPosition', 
                (overall_security, avg_privacy_score * 33.33),
                xytext=(10, 10), textcoords='offset points',
                bbox=dict(boxstyle='round,pad=0.3', facecolor='white', alpha=0.8),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0'))
else:
    ax3.text(0.5, 0.5, 'Insufficient data for trade-off analysis', ha='center', va='center',
             transform=ax3.transAxes, fontsize=14, alpha=0.5)
    ax3.set_title('Security vs Privacy Trade-off', fontweight='bold')

# 4. Compliance and Risk Assessment
compliance_data = {
    'GDPR Compliance': 85 if avg_privacy_score > 2 else 60,
    'HIPAA Compliance': 80 if overall_security > 80 else 55,
    'Data Protection': 90 if privacy_scores and max(privacy_scores) >= 2.5 else 65,
    'Encryption Strength': overall_security if overall_security > 0 else 75
}

compliance_names = list(compliance_data.keys())
compliance_scores = list(compliance_data.values())
colors = ['green' if score >= 80 else 'orange' if score >= 60 else 'red' for score in compliance_scores]

bars = ax4.barh(compliance_names, compliance_scores, color=colors, alpha=0.8)
ax4.set_title('Compliance and Risk Assessment', fontweight='bold')
ax4.set_xlabel('Compliance Score (%)')
ax4.set_xlim(0, 100)
ax4.grid(True, alpha=0.3, axis='x')

# Add value labels and risk levels
for bar, score in zip(bars, compliance_scores):
    risk_level = "Low Risk" if score >= 80 else "Medium Risk" if score >= 60 else "High Risk"
    ax4.text(bar.get_width() + 1, bar.get_y() + bar.get_height()/2,
             f'{score}% ({risk_level})', ha='left', va='center', fontweight='bold')

plt.tight_layout()
plt.show()

# Print detailed security and privacy analysis
print("🛡️ Security and Privacy Analysis:")
print("=" * 60)

print(f"🔒 Security Assessment:")
print(f"   Overall Security Score: {overall_security:.1f}/100")
print(f"   Attack Resistance Score: {attack_resistance:.1f}/100")

if overall_security >= 90:
    security_level = "Excellent - Production Ready"
elif overall_security >= 80:
    security_level = "Very Good - Suitable for most applications"
elif overall_security >= 70:
    security_level = "Good - Consider improvements for critical applications"
else:
    security_level = "Needs Improvement - Not recommended for production"

print(f"   Security Level: {security_level}")

if privacy_scores and privacy_names:
    print(f"\n🔐 Privacy Assessment:")
    for name, score in zip(privacy_names, privacy_scores):
        level = "High" if score >= 2.5 else "Medium" if score >= 1.5 else "Low"
        print(f"   {name}: {score:.2f} ({level})")
    
    avg_privacy = np.mean(privacy_scores)
    print(f"   Average Privacy Score: {avg_privacy:.2f}/3.0")

print(f"\n📋 Compliance Analysis:")
for name, score in compliance_data.items():
    status = "✅ Compliant" if score >= 80 else "⚠️ Partial" if score >= 60 else "❌ Non-Compliant"
    print(f"   {name}: {score}% {status}")

print(f"\n💡 Recommendations:")
recommendations = []

if overall_security < 80:
    recommendations.append("Improve FHE parameter security settings")
if avg_privacy_score < 2.0 if privacy_scores else True:
    recommendations.append("Enhance privacy protection mechanisms")
if min(compliance_scores) < 70:
    recommendations.append("Address compliance gaps for regulatory requirements")

if not recommendations:
    recommendations.append("Current configuration provides good security and privacy balance")

for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

## 5. Best Configuration Summary

Analyze and summarize the best configuration based on all evaluation metrics.

In [ ]:
# Best Configuration Analysis and Summary
print("⭐ BEST CONFIGURATION ANALYSIS")
print("=" * 60)

# Analyze all available configurations and metrics
best_config = {
    "performance": {},
    "privacy": {},
    "security": {},
    "recommendations": []
}

# Performance Analysis
perf_metrics = evaluation_data.get('performance_metrics', {})
if perf_metrics:
    best_config["performance"] = {
        "accuracy": perf_metrics.get('accuracy', 0),
        "f1_score": perf_metrics.get('f1_score', 0),
        "roc_auc": perf_metrics.get('roc_auc', 0),
        "overall_score": np.mean([
            perf_metrics.get('accuracy', 0),
            perf_metrics.get('precision', 0),
            perf_metrics.get('recall', 0),
            perf_metrics.get('f1_score', 0)
        ])
    }

# Privacy Analysis
privacy_utility = evaluation_data.get('privacy_utility_experiments', {})
if privacy_utility and 'epsilon_values' in privacy_utility:
    epsilon_vals = privacy_utility['epsilon_values']
    accuracy_vals = privacy_utility['accuracy_values']
    
    # Find optimal epsilon (best accuracy/privacy trade-off)
    if len(epsilon_vals) > 1 and len(accuracy_vals) > 1:
        # Calculate efficiency: accuracy per unit privacy cost (1/epsilon)
        efficiency_scores = [acc / eps for acc, eps in zip(accuracy_vals, epsilon_vals)]
        best_idx = np.argmax(efficiency_scores)
        
        best_config["privacy"] = {
            "optimal_epsilon": epsilon_vals[best_idx],
            "accuracy_at_optimal": accuracy_vals[best_idx],
            "efficiency_score": efficiency_scores[best_idx],
            "privacy_level": "Very High" if epsilon_vals[best_idx] <= 0.1 else 
                           "High" if epsilon_vals[best_idx] <= 1.0 else 
                           "Medium" if epsilon_vals[best_idx] <= 5.0 else "Low"
        }

# Security Analysis
security_metrics = evaluation_data.get('security_metrics', {})
if security_metrics:
    best_config["security"] = {
        "overall_score": security_metrics.get('overall_security', {}).get('overall_security_score', 0),
        "attack_resistance": security_metrics.get('attack_resistance', {}).get('overall_attack_resistance_score', 0),
        "key_strength": security_metrics.get('key_security', {}).get('key_security_strength', 0)
    }

# Model Comparison Analysis
model_comparison = evaluation_data.get('model_comparison', {})
best_model = None
best_model_score = 0

if model_comparison:
    for model_name, model_data in model_comparison.items():
        # Calculate composite score (weighted average of clear and FHE accuracy)
        clear_acc = model_data.get('clear_accuracy', 0)
        fhe_acc = model_data.get('fhe_accuracy', 0)
        composite_score = 0.6 * clear_acc + 0.4 * fhe_acc  # Weight clear accuracy higher
        
        if composite_score > best_model_score:
            best_model_score = composite_score
            best_model = {
                "name": model_name,
                "clear_accuracy": clear_acc,
                "fhe_accuracy": fhe_acc,
                "composite_score": composite_score,
                "accuracy_loss": clear_acc - fhe_acc
            }

# Generate comprehensive recommendations
recommendations = []

# Performance recommendations
if best_config["performance"]:
    overall_perf = best_config["performance"]["overall_score"]
    if overall_perf >= 0.85:
        recommendations.append("✅ Excellent performance - ready for production")
    elif overall_perf >= 0.75:
        recommendations.append("⚠️ Good performance - consider minor optimizations")
    else:
        recommendations.append("❌ Performance needs improvement - review model architecture")

# Privacy recommendations
if best_config["privacy"]:
    optimal_eps = best_config["privacy"]["optimal_epsilon"]
    if optimal_eps <= 1.0:
        recommendations.append("✅ Strong privacy protection with optimal ε")
    else:
        recommendations.append("⚠️ Consider reducing ε for stronger privacy")

# Security recommendations
if best_config["security"]:
    security_score = best_config["security"]["overall_score"]
    if security_score >= 85:
        recommendations.append("✅ High security level - suitable for sensitive data")
    elif security_score >= 70:
        recommendations.append("⚠️ Adequate security - monitor for improvements")
    else:
        recommendations.append("❌ Security needs enhancement")

# Model recommendations
if best_model:
    accuracy_loss = best_model["accuracy_loss"]
    if accuracy_loss <= 0.05:
        recommendations.append("✅ Minimal accuracy loss in FHE mode")
    elif accuracy_loss <= 0.10:
        recommendations.append("⚠️ Acceptable accuracy loss for privacy benefits")
    else:
        recommendations.append("❌ High accuracy loss - optimize FHE implementation")

best_config["recommendations"] = recommendations

# Create summary visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Best Configuration Summary Dashboard', fontsize=16, fontweight='bold')

# 1. Overall Scores Radar Chart
if all(key in best_config for key in ["performance", "security"]):
    categories = ['Performance', 'Privacy', 'Security', 'Efficiency']
    values = [
        best_config["performance"]["overall_score"] * 100,
        (best_config["privacy"]["efficiency_score"] * 10) if best_config["privacy"] else 70,
        best_config["security"]["overall_score"] if best_config["security"] else 70,
        best_model["composite_score"] * 100 if best_model else 70
    ]
    
    # Create radar chart
    angles = np.linspace(0, 2 * np.pi, len(categories), endpoint=False).tolist()
    values += values[:1]
    angles += angles[:1]
    
    ax1 = plt.subplot(2, 2, 1, projection='polar')
    ax1.plot(angles, values, 'o-', linewidth=3, color='#2E86AB', alpha=0.8)
    ax1.fill(angles, values, alpha=0.25, color='#2E86AB')
    ax1.set_xticks(angles[:-1])
    ax1.set_xticklabels(categories)
    ax1.set_ylim(0, 100)
    ax1.set_yticks([20, 40, 60, 80, 100])
    ax1.set_yticklabels(['20', '40', '60', '80', '100'])
    ax1.grid(True, alpha=0.3)
    ax1.set_title('Overall Configuration Score', fontweight='bold', pad=20)

# 2. Best Model Comparison
if best_model:
    model_metrics = ['Clear Accuracy', 'FHE Accuracy', 'Composite Score']
    model_values = [
        best_model["clear_accuracy"],
        best_model["fhe_accuracy"],
        best_model["composite_score"]
    ]
    
    bars = ax2.bar(model_metrics, model_values, color=['skyblue', 'lightcoral', 'lightgreen'], alpha=0.8)
    ax2.set_title(f'Best Model: {best_model["name"].replace("_", " ").title()}', fontweight='bold')
    ax2.set_ylabel('Score')
    ax2.set_ylim(0, 1.0)
    ax2.grid(True, alpha=0.3, axis='y')
    
    for bar, value in zip(bars, model_values):
        ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.01,
                 f'{value:.3f}', ha='center', va='bottom', fontweight='bold')
else:
    ax2.text(0.5, 0.5, 'No model comparison data', ha='center', va='center',
             transform=ax2.transAxes, fontsize=14, alpha=0.5)
    ax2.set_title('Best Model Analysis', fontweight='bold')

# 3. Privacy-Utility Optimization
if best_config["privacy"]:
    epsilon_range = np.logspace(-1, 1, 100)  # 0.1 to 10
    optimal_eps = best_config["privacy"]["optimal_epsilon"]
    optimal_acc = best_config["privacy"]["accuracy_at_optimal"]
    
    # Create theoretical curve
    theoretical_acc = 0.6 + 0.3 * np.log10(epsilon_range + 0.1) / np.log10(10.1)
    
    ax3.semilogx(epsilon_range, theoretical_acc, '--', alpha=0.5, color='gray', label='Theoretical')
    ax3.semilogx([optimal_eps], [optimal_acc], 'o', markersize=15, color='red', 
                label=f'Optimal (ε={optimal_eps})', zorder=5)
    
    # Add privacy-utility experiments if available
    if privacy_utility:
        ax3.semilogx(privacy_utility['epsilon_values'], privacy_utility['accuracy_values'], 
                    'o-', color='#2E86AB', alpha=0.8, label='Experimental')
    
    ax3.set_title('Privacy-Utility Optimization', fontweight='bold')
    ax3.set_xlabel('Epsilon (ε)')
    ax3.set_ylabel('Accuracy')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Annotate optimal point
    ax3.annotate(f'Optimal Point\nε={optimal_eps}\nAcc={optimal_acc:.3f}',
                xy=(optimal_eps, optimal_acc), xytext=(optimal_eps*3, optimal_acc-0.05),
                arrowprops=dict(arrowstyle='->', color='red'),
                bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.7))
else:
    ax3.text(0.5, 0.5, 'No privacy optimization data', ha='center', va='center',
             transform=ax3.transAxes, fontsize=14, alpha=0.5)
    ax3.set_title('Privacy-Utility Optimization', fontweight='bold')

# 4. Recommendation Summary
recommendation_text = "\n".join([f"• {rec}" for rec in recommendations[:6]])  # Limit to 6 recommendations
ax4.text(0.05, 0.95, "🎯 Key Recommendations:", transform=ax4.transAxes, 
         fontsize=14, fontweight='bold', va='top')
ax4.text(0.05, 0.85, recommendation_text, transform=ax4.transAxes, 
         fontsize=11, va='top', wrap=True)
ax4.set_xlim(0, 1)
ax4.set_ylim(0, 1)
ax4.axis('off')
ax4.set_title('Configuration Recommendations', fontweight='bold')

plt.tight_layout()
plt.show()

# Print comprehensive summary
print(f"🎯 PERFORMANCE SUMMARY:")
if best_config["performance"]:
    perf = best_config["performance"]
    print(f"   Overall Performance Score: {perf['overall_score']:.4f}")
    print(f"   Best Accuracy: {perf['accuracy']:.4f}")
    print(f"   Best F1-Score: {perf['f1_score']:.4f}")
    print(f"   Best ROC-AUC: {perf['roc_auc']:.4f}")

print(f"\n🔒 PRIVACY SUMMARY:")
if best_config["privacy"]:
    priv = best_config["privacy"]
    print(f"   Optimal Epsilon (ε): {priv['optimal_epsilon']}")
    print(f"   Accuracy at Optimal ε: {priv['accuracy_at_optimal']:.4f}")
    print(f"   Privacy Level: {priv['privacy_level']}")
    print(f"   Efficiency Score: {priv['efficiency_score']:.4f}")

print(f"\n🛡️ SECURITY SUMMARY:")
if best_config["security"]:
    sec = best_config["security"]
    print(f"   Overall Security Score: {sec['overall_score']:.1f}/100")
    print(f"   Attack Resistance: {sec['attack_resistance']:.1f}/100")
    print(f"   Key Strength: {sec['key_strength']} bits")

print(f"\n🤖 BEST MODEL:")
if best_model:
    print(f"   Model: {best_model['name'].replace('_', ' ').title()}")
    print(f"   Clear Accuracy: {best_model['clear_accuracy']:.4f}")
    print(f"   FHE Accuracy: {best_model['fhe_accuracy']:.4f}")
    print(f"   Accuracy Loss: {best_model['accuracy_loss']:.4f} ({best_model['accuracy_loss']/best_model['clear_accuracy']*100:.2f}%)")
    print(f"   Composite Score: {best_model['composite_score']:.4f}")

print(f"\n💡 KEY RECOMMENDATIONS:")
for i, rec in enumerate(recommendations, 1):
    print(f"   {i}. {rec}")

print(f"\n⭐ FINAL ASSESSMENT:")
overall_grade = "A" if (best_config["performance"]["overall_score"] >= 0.85 and 
                       best_config["security"]["overall_score"] >= 85) else \
               "B" if (best_config["performance"]["overall_score"] >= 0.75 and 
                       best_config["security"]["overall_score"] >= 75) else \
               "C" if (best_config["performance"]["overall_score"] >= 0.65) else "D"

print(f"   Overall Grade: {overall_grade}")
print(f"   Readiness: {'Production Ready' if overall_grade in ['A', 'B'] else 'Needs Improvement'}")
print(f"   Confidence: {'High' if overall_grade == 'A' else 'Medium' if overall_grade == 'B' else 'Low'}")

# Save best configuration to file
config_output = {
    "best_configuration": best_config,
    "best_model": best_model,
    "overall_grade": overall_grade,
    "timestamp": pd.Timestamp.now().isoformat()
}

output_file = Path('../data/results/best_configuration.json')
output_file.parent.mkdir(parents=True, exist_ok=True)

with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(config_output, f, indent=2, ensure_ascii=False)

print(f"\n💾 Best configuration saved to: {output_file}")
print(f"✨ Evaluation analysis completed successfully!")